In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from utils.context_data_csv import ContextDataCSV
from utils.matrix_optimizer import MatrixOptimizer
import utils.sulfur.constant as const

from oasis.functions import deconvolve

/app/OASIS/oasis/functions.py:13: UserWarning: Could not find cvxpy. Don't worry, you can still use OASIS, just not the slower interior point methods we compared to in the papers.
  warn("Could not find cvxpy. Don't worry, you can still use OASIS, " +


In [2]:
all_shock_cells = {
    'ID181106CreA': ['C261', 'C305', 'C347', 'C349', 'C351', 'C371'],
    'ID181106CreB': ['C058', 'C062', 'C089', 'C126', 'C138', 'C172'],
    'ID181106CreC': ['C017', 'C051', 'C065', 'C112', 'C117', 'C203', 'C214', 'C227'],
    'ID181106CreG': ['C127', 'C239', 'C270', 'C323', 'C347', 'C350', 'C399', 'C402'],
    'ID181106CreH': ['C164', 'C180', 'C184', 'C206', 'C243', 'C253', 'C298', 'C327', 'C351', 'C534', 'C542'],
    'ID181106CreI': ['C136', 'C197', 'C273', 'C292', 'C294', 'C306', 'C333'],
    'ID181106CreK': ['C083', 'C088', 'C100', 'C118', 'C137', 'C149', 'C203', 'C390'],
    'ID181106CreL': ['C200'],
    'ID181106CreN': ['C199', 'C255', 'C258', 'C273', 'C334'],
    'ID181106CreQ': ['C235', 'C393']
}

In [3]:
index = const.ANIMAL_NAMES
columns = const.CONTEXTS
event_rate_for_1sd_shock_cells = pd.DataFrame(index=index, columns=columns)

for animal_name in const.ANIMAL_NAMES:
    print('animal name: %s' % animal_name)
    
    shock_cells = all_shock_cells[animal_name]
    
    for context_name in const.CONTEXTS:
        print('context: %s' % context_name)
        
        context_data = ContextDataCSV(animal_name, context_name)
        df = context_data.data_frame
        df = MatrixOptimizer(df).divide_sd()
        
        seconds = len(df) / 10.0
        
        event_rates = np.array([])
        shock_cells_df = df.loc[:, shock_cells]
        for cell_name, values in shock_cells_df.iteritems():
            if values.isnull().all() or (values == 0).all():
                print('%s is nan' % cell_name)
                continue
                
            values.fillna(0.0, inplace=True)
            _c, spikes, _b, _g, _lam = deconvolve(values.to_numpy(), g=(None, None), penalty=1)
            
            event_rate = len(spikes[spikes >= 0.05]) / seconds
            event_rates = np.append(event_rates, event_rate)
            
        event_rate_for_1sd_shock_cells.loc[animal_name, context_name] = np.mean(event_rates)
        
event_rate_for_1sd_shock_cells

animal name: ID181106CreA
context: HC1


FileNotFoundError: [Errno 2] No such file or directory: './resources/context_data/ID181106CreA/HC1.csv'

In [ ]:
event_rate_for_1sd_shock_cells.mean()

In [ ]:
all_event_rate_df = pd.read_csv('./resources/event_rates/oasis_by_cells/all_event_rates_threshold005.csv', index_col=[0], header=[0])
engram_event_rate_df = pd.read_csv('./resources/event_rates/oasis_by_cells/engram_event_rates_threshold005.csv', index_col=[0], header=[0])
non_engram_event_rate_df = pd.read_csv('./resources/event_rates/oasis_by_cells/non_engram_event_rates_threshold005.csv', index_col=[0], header=[0])

In [ ]:
all_event_rate_df.mean()

In [ ]:
engram_event_rate_df.mean()

In [ ]:
non_engram_event_rate_df.mean()

In [ ]:
x = all_event_rate_df.columns
y = all_event_rate_df.mean()
error = all_event_rate_df.sem()

engram_x = engram_event_rate_df.columns
engram_y = engram_event_rate_df.mean()
engram_err = engram_event_rate_df.sem()

non_engram_x = non_engram_event_rate_df.columns
non_engram_y = non_engram_event_rate_df.mean()
non_engram_err = non_engram_event_rate_df.sem()

shock_cell_x = event_rate_for_1sd_shock_cells.columns
shock_cell_y = event_rate_for_1sd_shock_cells.mean()
shock_cell_err = event_rate_for_1sd_shock_cells.sem()

fig, axes = plt.subplots(2, figsize=(8, 8), dpi=200, tight_layout=True, sharex='all')

axes[0].errorbar(x, y, yerr=error, capsize=2)
axes[0].plot(x, y, marker="s", markersize=8, markeredgewidth=3, label='all cell', color='royalblue')

axes[0].errorbar(shock_cell_x, shock_cell_y, yerr=shock_cell_err, capsize=2, color='yellowgreen')
axes[0].plot(shock_cell_x, shock_cell_y, marker="s", markersize=8, markeredgewidth=3, label='shock cell', color='yellowgreen')

axes[0].set_ylim(0, 2.0)
axes[0].set_ylabel('Ca2+ Event Rate (Hz)')
axes[0].legend(loc='lower left')
#aix[0].set_title('Ca2+ event rates (threshold: 0.05)')

axes[1].errorbar(engram_x, engram_y, yerr=engram_err, capsize=2)
axes[1].plot(engram_x, engram_y, marker="s", markersize=8, markeredgewidth=3, label='engram cell', color='royalblue')

axes[1].errorbar(non_engram_x, non_engram_y, yerr=non_engram_err, capsize=2)
axes[1].plot(non_engram_x, non_engram_y, marker="s", markersize=8, markeredgewidth=3, label='non-engram cell', color='tomato')

axes[1].errorbar(shock_cell_x, shock_cell_y, yerr=shock_cell_err, capsize=2, color='yellowgreen')
axes[1].plot(shock_cell_x, shock_cell_y, marker="s", markersize=8, markeredgewidth=3, label='shock cell', color='yellowgreen')

axes[1].set_ylim(0, 2.0)
axes[1].set_ylabel('Ca2+ Event Rate (Hz)')
axes[1].legend(loc='lower left')

plt.xticks(rotation=45)

plt.show()

In [ ]:
all_shock_cells = {
    'ID181106CreA': ['C261', 'C305', 'C347', 'C349', 'C351'],
    'ID181106CreB': ['C058', 'C089', 'C126'],
    'ID181106CreC': ['C017', 'C065', 'C112', 'C117', 'C203', 'C227'],
    'ID181106CreG': ['C127', 'C239', 'C323', 'C347', 'C350', 'C399', 'C402'],
    'ID181106CreH': ['C164', 'C180', 'C184', 'C243', 'C253', 'C298', 'C327', 'C351', 'C534', 'C542'],
    'ID181106CreI': ['C136', 'C197', 'C273', 'C292', 'C294', 'C306', 'C333'],
    'ID181106CreK': ['C083', 'C088', 'C100', 'C149', 'C390'],
    'ID181106CreL': ['C200'],
    'ID181106CreN': ['C199', 'C255', 'C258', 'C273'],
    'ID181106CreQ': ['C393']
}

In [ ]:
index = const.ANIMAL_NAMES
columns = const.CONTEXTS
event_rate_for_2sd_shock_cells = pd.DataFrame(index=index, columns=columns)

for animal_name in const.ANIMAL_NAMES:
    print('animal name: %s' % animal_name)
    
    shock_cells = all_shock_cells[animal_name]
    
    for context_name in const.CONTEXTS:
        print('context: %s' % context_name)
        
        context_data = ContextDataCSV(animal_name, context_name)
        df = context_data.data_frame
        df = MatrixOptimizer(df).divide_sd()
        
        seconds = len(df) / 10.0
        
        event_rates = np.array([])
        shock_cells_df = df.loc[:, shock_cells]
        for cell_name, values in shock_cells_df.iteritems():
            if values.isnull().all() or (values == 0).all():
                print('%s is nan' % cell_name)
                continue
                
            values.fillna(0.0, inplace=True)
            _c, spikes, _b, _g, _lam = deconvolve(values.to_numpy(), g=(None, None), penalty=1)
            
            event_rate = len(spikes[spikes >= 0.05]) / seconds
            event_rates = np.append(event_rates, event_rate)
            
        event_rate_for_2sd_shock_cells.loc[animal_name, context_name] = np.mean(event_rates)
        
event_rate_for_2sd_shock_cells

In [ ]:
x = all_event_rate_df.columns
y = all_event_rate_df.mean()
error = all_event_rate_df.sem()

engram_x = engram_event_rate_df.columns
engram_y = engram_event_rate_df.mean()
engram_err = engram_event_rate_df.sem()

non_engram_x = non_engram_event_rate_df.columns
non_engram_y = non_engram_event_rate_df.mean()
non_engram_err = non_engram_event_rate_df.sem()

shock_cell_x = event_rate_for_2sd_shock_cells.columns
shock_cell_y = event_rate_for_2sd_shock_cells.mean()
shock_cell_err = event_rate_for_2sd_shock_cells.sem()

fig, axes = plt.subplots(3, figsize=(8, 10), dpi=200, tight_layout=True)

axes[0].errorbar(x, y, yerr=error, capsize=2, color='dimgray')
axes[0].plot(x, y, marker="s", markersize=8, markeredgewidth=3, label='all cell', color='dimgray')

axes[0].errorbar(shock_cell_x, shock_cell_y, yerr=shock_cell_err, capsize=2, color='yellowgreen')
axes[0].plot(shock_cell_x, shock_cell_y, marker="o", markersize=8, markeredgewidth=3, label='shock cell', color='yellowgreen')

axes[0].set_ylim(0, 2.0)
axes[0].set_ylabel('Ca2+ Event Rate (Hz)')
axes[0].legend(loc='lower left')
axes[0].set_xticklabels(x, rotation=45)

axes[1].errorbar(engram_x, engram_y, yerr=engram_err, capsize=2)
axes[1].plot(engram_x, engram_y, marker="s", markersize=8, markeredgewidth=3, label='engram cell', color='royalblue')

axes[1].errorbar(shock_cell_x, shock_cell_y, yerr=shock_cell_err, capsize=2, color='yellowgreen')
axes[1].plot(shock_cell_x, shock_cell_y, marker="o", markersize=8, markeredgewidth=3, label='shock cell', color='yellowgreen')

axes[1].set_ylim(0, 2.0)
axes[1].set_ylabel('Ca2+ Event Rate (Hz)')
axes[1].legend(loc='lower left')
axes[1].set_xticklabels(x, rotation=45)

axes[2].errorbar(non_engram_x, non_engram_y, yerr=non_engram_err, capsize=2, color='tomato')
axes[2].plot(non_engram_x, non_engram_y, marker="s", markersize=8, markeredgewidth=3, label='non-engram cell', color='tomato')

axes[2].errorbar(shock_cell_x, shock_cell_y, yerr=shock_cell_err, capsize=2, color='yellowgreen')
axes[2].plot(shock_cell_x, shock_cell_y, marker="o", markersize=8, markeredgewidth=3, label='shock cell', color='yellowgreen')

axes[2].set_ylim(0, 2.0)
axes[2].set_ylabel('Ca2+ Event Rate (Hz)')
axes[2].legend(loc='lower left')
axes[2].set_xticklabels(x, rotation=45)

plt.xticks(rotation=45)

plt.show()

In [ ]:
event_rate_for_2sd_shock_cells.mean()

In [ ]:
event_rate_for_2sd_shock_cells.to_csv('./resources/event_rates/oasis_by_cells/2sd_shock_cell_event_rates.csv')